In [2]:
import sympy
sympy.init_printing(use_latex='mathjax')
import numpy as np

## Pivot columns

<p>Determine which columns of the following matrix are pivot colummns.</p>  

$$
\begin{bmatrix}
1 & 3 & 2 & 3 & 4 \\
2 & 6 & 1 & 3 & 3 \\
1 & 3 & 3 & 4 & 6 \\
1 & 3 & 1 & 2 & 1 \\
\end{bmatrix}
$$

In [45]:
A = Matrix([[1, 3, 2, 3, 4],
           [2, 6, 1, 3, 3],
           [1, 3, 3, 4, 6],
           [1, 3, 1, 2, 1]]
          )
A.rref()

⎛⎡1  3  0  1  0⎤           ⎞
⎜⎢             ⎥           ⎟
⎜⎢0  0  1  1  0⎥           ⎟
⎜⎢             ⎥, (0, 2, 4)⎟
⎜⎢0  0  0  0  1⎥           ⎟
⎜⎢             ⎥           ⎟
⎝⎣0  0  0  0  0⎦           ⎠

## Vectors in Span

<p>Consider the following vectors:
$$
{\mathbf v}_1 = 
\begin{bmatrix}
\phantom{-}1\\
\phantom{-}2\\
\phantom{-}3\\
\end{bmatrix}
\hskip 4mm
{\mathbf v}_2 = 
\begin{bmatrix}
\phantom{-}2\\
\phantom{-}2\\
\phantom{-}3\\
\end{bmatrix}
\hskip 4mm
{\mathbf v}_3 = 
\begin{bmatrix}
\phantom{-}1\\
-2\\
-3\\
\end{bmatrix}
$$
</p>
<p>
Determine which of the following vectors are in \(\text{Span}({\mathbf v}_1, {\mathbf v}_2, {\mathbf v}_3)\).

$$
{\mathbf w}_1 = 
\begin{bmatrix}
-2\\
-2\\
-3\\
\end{bmatrix}
\hskip 4mm
{\mathbf w}_2 = 
\begin{bmatrix}
\phantom{-}0\\
\phantom{-}1\\
\phantom{-}0\\
\end{bmatrix}
\hskip 4mm
{\mathbf w}_3 = 
\begin{bmatrix}
\phantom{-}1\\
\phantom{-}0\\
\phantom{-}0\\
\end{bmatrix}
\hskip 4mm
{\mathbf w}_4 = 
\begin{bmatrix}
\phantom{-}0\\
\phantom{-}0\\
\phantom{-}0\\
\end{bmatrix}
\hskip 4mm
{\mathbf w}_5 = 
\begin{bmatrix}
-1\\
\phantom{-}1\\
\phantom{-}1\\
\end{bmatrix}
\hskip 4mm
{\mathbf w}_6 = 
\begin{bmatrix}
\phantom{-}5\\
\phantom{-}2\\
\phantom{-}3\\
\end{bmatrix}
$$   

</p>

## Solutions of linear equations

In [29]:
import math
class FormatError(Exception):
    pass

def scrub_string(s, glob_dir={}, loc_dir={}, clear=False):
    """
    Scrubs a string, evaluates it, and returns the
    evaluated expression. It also replaces "^" with "**" in the string.

    :s:
        A string.
    :glob_dir:
        A directory of global objects permitted in the string
    :local_dir:
        A directory of local objects names permitted in the string
    :clear: Boolean.
        If False all objects from the math module as well as the abs
        and pow functions are inserted in glob_dir.

    Returns:
        The evaluated string.
    """

    if not clear:
        allowed_builtins = {"abs":abs, "pow":pow, }
        math_dir = {s:getattr(math, s) for s in dir(math) if "__" not in s}
        glob_dir["__builtins__"] = allowed_builtins
        glob_dir.update(math_dir)

    # disable double underscores since can be used to get object attributes
    if "__" in s:
        raise FormatError("Invalid input")

    try:
        s.strip()
        s.replace(" ", "")
        s = s.replace("^", "**")
        return eval(s ,glob_dir, loc_dir)
    except Exception:
        raise FormatError("Invalid input")





import sympy
import numpy as np

def linear_eqs_grader(A, b, symb = "x", atol=0.1, rtol=0):
    
    A = sympy.Matrix(A)
    b = sympy.Matrix(b)
    n = A.shape[1]
    Ab = A.col_insert(n, b)
    no_solutions = (n in Ab.rref()[1])
    rank = len(A.rref()[1])
    nullity = n - rank
    
    
    # columns of the test matrix give test values of free variables 
    # for checking the solution
    if nullity > 0:
        test_matrix = sympy.eye(nullity).col_insert(0, sympy.Matrix([0]*nullity))
    else:
        test_matrix = sympy.Matrix([0])
        
    # prepare a disctionary of names of variables appearing in the system of equations 
    # and their associated sympy symbols; this is used for parsing answer strings
    variables = {}
    for i in range(n):
        variables["{0}_{1}".format(symb, i+1)] = sympy.Symbol("{0}_{1}".format(symb, i+1))
    
    # convert the coefficient matrix and the vector of constants into 
    # numpy array - it is more convenient to test the solution in this form
    A_arr = np.array(A).astype(float)
    b_arr = np.array(b).astype(float).ravel()
    
    def check_answer(expect, ans):
        
        # test for no solutions
        ans_str = [s.strip().lower() for s in ans]
        if "none" in ans_str:
            if no_solutions:
                return True
            else:
                return False
        
        # number of entries in the answer must match the number of variables. 
        if len(ans) != n:
            return False
        
        # convert answer entries into sympy expressions 
        # and collect free variables appearing in the answer
        free_vars = set()
        eval_ans = []
        

        for i in range(n):
            y = scrub_string(ans[i], glob_dir=variables)
            try :
                fy = y.free_symbols
            except AttributeError:
                fy = set()
            free_vars.update(fy)
            eval_ans.append(y)
        
        free_vars = list(free_vars)
        
        # the number of free variables in the answer must match the nullity 
        # of the coefficient matrix
        if len(free_vars) != nullity:
            return False
         
        # plug in test values into free variables in the answer
        # check is the resulting vector is a solution of the system
        for i in range(nullity+1):
            test_vals = test_matrix[:, i]
            test_sub = list(zip(free_vars, test_vals))
            ans_subs = []
            for y in eval_ans:
                try:
                    ys = y.subs(test_sub)
                except AttributeError:
                    ys = y
                ans_subs.append(ys)
            ans_subs_arr = np.array(ans_subs).astype(float)
            ans_prod = np.dot(A_arr, ans_subs_arr)
            if not np.allclose(ans_prod, b_arr, atol= atol, rtol=rtol):
                return False
        
        return True
    
    return check_answer

#### Testing

In [30]:
A = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
b = [1, 2, 3]
ans = ["none", "2", "3"]
foo = linear_eqs_grader(A, b)
print(foo(expect=None, ans=ans))

False


In [31]:
A = [[0, 0, 0, 1, -1],
     [1, 1, 0, 0, 0],
     [1, 0, 1, 0, 0],
     [0, 1, -1,1, 0],
     [0, 0, 0, 0, 1]
    ]
b = [35, 85, 75, 70, 25]
b = [35, 85, 75, 70, 25]
ans = ["85-x_2", "x_2", "x_2-10", "60", "25"]
foo = linear_eqs_grader(A, b)
print(foo(expect=None, ans=ans))

True


In [32]:
A = [[-1,  1,  1,  0,  0,  0],
             [ 1,  0,  0,  1,  0,  0],
             [ 0, -1,  0,  0,  0,  1],
             [ 0,  0,  0,  0, -1, -1],
             [ 0,  0, -1, -1,  1,  0],
            ]
b = [100, 80, -30, -100, -50]
ans = ["80-x_4", "30+x_6", "150-x_4-x_6", "x_4", "100-x_6", "x_6"]
foo = linear_eqs_grader(A, b)
print(foo(expect=None, ans=ans))

True


In [33]:
import sympy

A = sympy.Matrix([
[ -1,  1, 1, 0, 0, 0], 
[  1,  0, 0, 1, 0, 0],
[  0, -1, 0, 0, 0, 1],
[  0,  0, 0, 0,-1,-1],
[  0,  0,-1,-1, 1, 0]])

b = sympy.Matrix([100, 80, -30, -100, -50]) 
ans = ["80-x_4", "30+x_6", "150-x_4 - x_6", "x_4", "100 - x_6", "x_6"]
foo = linear_eqs_grader(A, b)
print(foo(expect=None, ans=ans))

True


In [44]:
A = [[1, 3,   5, -2],
     [2, 7,   3,  1],
     [1, 5,  -9,  8],
     [5, 18,  4,  5]]   
  
b = [3, 5, 1, 12]  

ans = ["6- 26*x_3 + 17*x_4", "-1 + 7*x_3 - 5*x_4", "x_3", "x_4"] 
foo = linear_eqs_grader(A, b)
print(foo(expect=None, ans=ans))

True


In [39]:
sol = ["-2", "1", "4", "3"]
foo = linear_eqs_grader(A, b)
print(foo(expect=None, ans=sol))

True


In [2]:
x = [ u'none', u'', u'', u'', u'', u'' ]

In [3]:
"none" in x

True

In [4]:
import graders309 as gr  
  
gr.np_randomizer()  
    
B = gr.random_int_matrix(m=3, n=3, matrix_min=-3, matrix_max=4, invertible=True)
v, c = gr.random_int_combination(B, coeff_min = -6, coeff_max = 7)

b11, b12, b13 = [int(b) for b in B[0]]
b21, b22, b23 = [int(b) for b in B[1]]
b31, b32, b33 = [int(b) for b in B[2]]

ssc = str(c)
sB = str(B)
sv = str(v)
v1, v2, v3 = [int(y) for y in v]
c1, c2, c3 = [int(y) for y in c]
  
w1, w2 = 1, 2
s = int(sum(c))
sw1 = s*w1
sw2 = s*w2
sc = c1 + c2 + c3  
  
atol = 0.01
rtol = 0
  
def check_answer(expect, ans):
    try:
        x = eval("np.array(" + ans + ").astype(float)")
    except Exception: 
        return False
    if x.shape != (2,):
        return False
    if np.allclose(x, sum(c)*np.array([w1, w2]), atol=atol, rtol=rtol): 
        return True
    #raise Exception("x={}, ans={}, v={}, B={}".format(x, (c1+c2+c3)*np.array([w1, w2]), v, B))
    return False

In [5]:
B

array([[ 4,  2, -1],
       [ 0,  1, -3],
       [-1, -3, -2]])

In [6]:
v

array([-7, -8, 12])

In [7]:
c

array([ 1, -5,  1])

In [10]:
import numpy as np
A = np.arange(9).reshape(3,3)

In [11]:
A

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [12]:
b = np.array([10, 20, 30])

In [22]:
import graders309 as gr
  
import sympy
import numpy as np

V = gr.random_int_matrix(m=3, n=3, matrix_min=-3, matrix_max=3, invertible=True, max_det=3)
L = gr.random_int_matrix(m=2, n=1, matrix_min=-5, matrix_max=5).ravel()
v1 = V[:, 0]
v2 = V[:, 1]
v3 = V[:, 2]
L1 = L[0]
L2 = L[1]
LL = [L1, L1, L2]
VLL = V*np.array(LL)

MV = sympy.Matrix(V)
MLL = sympy.diag(*LL)
print(MV, MLL)
Msol = MV*MLL*MV**(-1)

sol = gr.matrix2str(Msol)
print("sol= ", sol)  
atol = 0.01
rtol = 0


def check_answer(expect, ans):
  
    A = str_2_array(ans)
    if A.shape != (3, 3):
        return False

    AV = np.dot(A, V)
    if not np.allclose(AV, VLL, atol= atol, rtol=rtol):
        return False
    else:
        return True

Matrix([[1, 1, -2], [1, 2, -3], [3, 1, -2]]) Matrix([[-5, 0, 0], [0, -5, 0], [0, 0, -2]])
sol=  [[10, -6, -3], [45/2, -14, -9/2], [15, -6, -8]]


In [17]:
sympy.diag([1, 2, 3])

Matrix([
[1],
[2],
[3]])